In [1]:
import discogs_client as DiscogsClient
from rapidfuzz import process as r_process, fuzz as r_fuzz
from rich import print as rprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from music_db_finder.fuzz_helpers import obj_ratio

import dotenv
env_file_name = '../../../.env'
dotenv.load_dotenv(env_file_name, override=True)

SPOTIPY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
DISCOGS_USER_TOKEN = os.getenv('DISCOGS_USER_TOKEN')
YTMUSIC_REFRESH_TOKEN = os.getenv('YTMUSIC_REFRESH_TOKEN')
YTMUSIC_ACCESS_TOKEN = os.getenv('YTMUSIC_ACCESS_TOKEN')

print(SPOTIPY_CLIENT_SECRET)

discogs_client = DiscogsClient.Client('ExampleApplication/0.1', user_token=DISCOGS_USER_TOKEN)
spotify_auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
spotify_client = spotipy.Spotify(client_credentials_manager=spotify_auth_manager)

70b24b83b00e48da8d2adfb637422206


In [14]:
def match_exact_discog(search_string, results):
    candidate = None
    for result in results:
        res_lower = getattr(result, 'name', getattr(result, 'title', None))
        if not res_lower:
            continue
        names = [res_lower.lower(), res_lower.lower().rsplit(maxsplit=1)[0]]
        if search_string.lower() in names:
            if not candidate:
                candidate = result
            else:
                return None
    return candidate

def find_band_exact(band_name):
    """Returns the artist object for searches that have only one exact match, 
    ignoring case. If multiple results are returned, returns None.
    
    It works by iterating through ALL the results and comparing search string to name
    and truncated name (one split from the right, for (1) duplicated names). If it
    finds two of the same string it exists with None.
    """
    # DONE optimization: iterate over results until two are found
    # if only 1 exists, it goes through it all, as it does now
    # if 2+ exist, it goes until it founds second
    # TODO optimization: use as chained, returning status and result
    # if only 1 exist return true and the object as a list
    # if multiple exists return false and all candidates as a list
    ### Those are contradicting functionalities, enable a switch?
    # Not required - the first one if generally faster:
    # If you want to get just the artist and it exist - you need to iterate all.
    # If you want to get just the artist and multiple exist:
    #   you have to iterate again through the next function, through all results
    #   or you give up
    # For first to be slower it needs to iterate through the list twice:
    #   that is, the second same artist must be at the very end
    #   and we can rely on search optimization that they push similar artists ahead
    # DONE: check if you've iterated through them all - for _ in result iterates all
    band_results = discogs_client.search(band_name, type='artist')
    candidate = None
    for result in band_results:
        names = [(res_lower := result.name.lower()), res_lower.rsplit(maxsplit=1)[0]]
        if band_name.lower() in names:
            if not candidate:
                candidate = result
            else:
                return None
    return candidate

def find_exact(search_string, search_type):
    results = discogs_client.search(search_string, type=search_type)
    return match_exact_discog(search_string, results)

def find_band_fuzzy(band_name, scorer=obj_ratio):
    band_results = discogs_client.search(band_name, type='artist')
    kwargs = {'scorer':scorer, 'limit':None, 'score_cutoff':75}
    fuzzy_results = r_process.extract(band_name, band_results, **kwargs)
    return map(lambda x: x[0], fuzzy_results)

def find_fuzzy(search_string, search_type, scorer=obj_ratio):
    results = discogs_client.search(search_string, type='artist')
    kwargs = {'scorer':scorer, 'limit':None, 'score_cutoff':75}
    fuzzy_results = r_process.extract(search_string, results, **kwargs)
    return map(lambda x: x[0], fuzzy_results)

def find_band_by_album(band_name, album_name, scorer=obj_ratio):
    bands = find_fuzzy(band_name, 'artist', scorer)
    for band in bands:
        print(band.name)
        print(band.releases)
        # result = find_exact(album_name, 'master')


In [103]:
def match_exact_discog(search_string, results):
    candidate = None
    for result in results:
        res_lower = getattr(result, 'name', getattr(getattr(result, 'master', getattr(result, 'main_release', 'none')), 'title', 'none'))
        if not res_lower:
            continue
        names = [res_lower.lower(), res_lower.lower().rsplit(maxsplit=1)[0]]
        if search_string.lower() in names:
            if not candidate:
                candidate = result
            else:
                return None
    return candidate

search_string = 'floodland'
search_type = 'master'
results = discogs_client.search(search_string, type=search_type)
print(results[0])
match_exact_discog(search_string, results)

# search_string = 'floodland'
# # release
# search_type = 'release'  # discogs_client.models.Release  # 313899   # 2836
# results = discogs_client.search(search_string, type=search_type)

# one = results[0]
# # print(one.master.title)
# print(getattr(one, 'name', getattr(getattr(one, 'master', getattr(one, 'main_release', 'none')), 'title', 'none')))

# # master
# search_type = 'master'  # discogs_client.models.Master  # 2836
# results = discogs_client.search(search_string, type=search_type)
# one = results[0]
# # print(one.main_release.title)
# print(getattr(one, 'name', getattr(getattr(one, 'master', getattr(one, 'main_release', 'none')), 'title', 'none')))


# search_string = 'the sisters of mercy'
# # release
# search_type = 'artist'
# results = discogs_client.search(search_string, type=search_type)

# one = results[0]
# # print(one.name)
# print(getattr(one, 'name', getattr(getattr(one, 'master', getattr(one, 'main_release', 'none')), 'title', 'none')))



<Master 2836 'The Sisters Of Mercy - Floodland'>


In [ ]:
def find_band_by_album(band_name, album_name, scorer=obj_ratio):
    bands = find_fuzzy(band_name, 'artist', scorer)
    for band in bands:
        print(band.name)
        print(band.id)
        try:
            for release in band.releases:
                print(release.title)
        except:
            print(f'Found error for artist {band.id}: {band.name} [discogs.com{band.url}]')
        # result = find_exact(album_name, 'master')

find_band_by_album('sisters of mercy', 'floodland')

In [36]:
band_name = 'sisters of mercy'
print(discogs_client.search(band_name, type='artist'))

find = list(find_fuzzy(band_name, 'artist'))
band = find[1]
print(band.releases)


# match_exact_discog('floodland', band.releases)
for release in band.releases:
    if 'floodland' in release.title.lower():
        print(release)

# dir(band)
# unofficial albums?!

<Master 2836 'Floodland'>
<Master 2836 'Floodland'>
<Master 1284386 "Floodland Demo's">
<Master 2095744 'Floodland Demos'>


In [ ]:
band_name = 'the sisters of mercy'
bands = list(find_band_fuzzy(band_name))
print(bands)
band = bands[0]
print(band.name)
print(band.releases)
print(list(band.releases))
dir(band)

In [31]:
band_name = 'the sisters of mercy'
album_name = 'floodland'
search_type = 'artist'
# print(list(find_band_fuzzy(band_name)))
# print(list(find_fuzzy(band_name, search_type)))

list(discogs_client.search(album_name, type='master'))

[<Master 2836 'The Sisters Of Mercy - Floodland'>,
 <Master 1098759 'Oscar Mulero - Floodland EP'>,
 <Master 2095744 'The Sisters Of Mercy - Floodland Demos'>,
 <Master 1284386 "The Sisters Of Mercy - Floodland Demo's">,
 <Master 217391 'Floodland - Ocean Of The Lost'>,
 <Master 217392 'Floodland - Decay'>,
 <Master 430797 'Aun (2) - Full Circle'>,
 <Master 1693442 'Analytica - Analytica'>,
 <Master 1238410 'Oscar Mulero - Pro-Files'>,
 <Master 1380655 'Planning For Burial - Quietly'>,
 <Master 1704501 'Zanias - Harmaline'>,
 <Master 1039749 'Oscar Mulero - In Bad Company'>,
 <Master 92738 'Rue East - Indoor Culture Remixes - Part 2'>,
 <Master 2913841 'Thee Flanders - Lockdown EP'>,
 <Master 483349 'Ektoise - Distortions'>,
 <Master 613531 'Various - Battery: A Tribute To Rammstein'>,
 <Master 286772 'Various - London v Madrid EP'>,
 <Master 1078543 'Ego Likeness - The Compass EPs'>,
 <Master 404719 'DSTR* - Silent World EP'>,
 <Master 3397648 'Zanias - Ecdysis'>,
 <Master 588254 'The